In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
df = pd.read_csv(os.path.join(dirname, filename))
df.head()

In [ ]:
# check the type of each column
df.dtypes

There are no numerical values, it means that we have to transform them to numerical values later.

In [ ]:
# check missing values :
df.isna().values.any()

There is no missing value, so we are good.


In [ ]:
# Transform the categorical variables into numerical values.
lb_make = LabelEncoder()
df["Mashroom_Cat"] = lb_make.fit_transform(df["class"])

In [ ]:
# Get rid of the original class
df = df.iloc[:,1:]
df.head()

Now the class is transform into 0/1s in the end of the columns

In [ ]:
# Split our data to features and target
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [ ]:
# Use get_dummies functions from pandas to transform the features to numerical values as well
X = pd.get_dummies(X,dummy_na=True)
X.head()

In [ ]:
# Split to train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Try to find the best algo, with the help of Pipeline
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]

for classifier in classifiers:
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))

They seem all good, except the SVC model. Let's try Random Forest, and find the best parameters.


In [ ]:
param_grid = { 
    'classifier__n_estimators': [200, 500],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth' : [4,5,6,7,8],
    'classifier__criterion' :['gini', 'entropy']}

rf = Pipeline(steps=[('classifier', RandomForestClassifier())])
CV = GridSearchCV(rf, param_grid, n_jobs= 1)
                  
CV.fit(X_train, y_train)  
print(CV.best_params_)    
print(CV.best_score_)

In [ ]:
# Using the best parameters and try with the Kfold, the accuracy is still 1.
seed = 3
kfold = KFold(10, random_state=seed,shuffle=True)
results = cross_val_score(CV.best_estimator_, X_train, y_train, cv=kfold)
print("Accuracy: %0.2f (+/- %0.2f)" % (results.mean(), results.std() * 2))

In [ ]:
# Predict with the best classifier 
CV.best_estimator_.fit(X_train, y_train)
y_pred = CV.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred,target_names=["e","p"]))